In [1]:
import sys
sys.path.append('../')
sys.path.append('../datamodules')
sys.path.append('../../')
import json
import os
import logging
from omegaconf import OmegaConf
from omegaconf.dictconfig import DictConfig
import torch
from datamodules import REGISTRY as DATAMODULE_REGISTRY
from datamodules import BaseDataModule
from rich.progress import track
from src.utils import load_pretrained_d2v_model

logger = logging.getLogger(__name__)

In [2]:
cfg_dict = {
    '_name': 'openwebtext',
    'model_state_dict': 'nlp_base.pt',
    'data_path': '../data',
    'out_path': '../data',
    'batch_size': 256,
    'num_workers': 5,
    'shuffle': False,
    'drop_last': False,
    'num_max_bpe_tokens': 512,
    'sample_break_mode': 'none',
}

In [3]:
cfg = OmegaConf.create(cfg_dict)

In [4]:
def extract_targets(cfg: DictConfig) -> None:
    model = load_pretrained_d2v_model(state_dict_path=os.path.join('..', 'models', cfg.model_state_dict))

    model.eval()
    
    datamodule_kwargs = OmegaConf.to_container(cfg)
    datamodule_name = datamodule_kwargs.pop('_name', None)
    datamodule_kwargs.pop('model_state_dict', None)
    
    datamodule_cls = DATAMODULE_REGISTRY[datamodule_name]
    
    datamodule:BaseDataModule = datamodule_cls(**datamodule_kwargs)

    logger.info('Setting up dataset and dataloader...')

    datamodule.prepare_data()

    datamodule.setup(stage='fit')

    train_dataloader = datamodule.train_dataloader()

    dir_name = f'kd_{datamodule_name}'

    kd_targets_path = os.path.join(cfg.out_path, dir_name)

    os.makedirs(kd_targets_path, exist_ok=True)

    index_items = []
    raise Exception
    with torch.no_grad():
        for idx, batch in track(enumerate(train_dataloader), description="Running predictions...", total=len(train_dataloader)):
            key = batch['modes'][0].name.lower() if key is None else key

            padding_mask = batch['padding_mask'] if 'padding_mask' in batch else None 
            pred = model.extract_features(
                source=batch[key],
                mode=None, # determined automatically in model
                padding_mask=padding_mask,
                mask=False, # we are creating targets from a teacher model for the student model, so no mask
                remove_extra_tokens=False,
            )
            filename = f'{idx}_{batch["id"][0]}-{batch["id"][-1]}.pt'
            index_items.append({
                "path": os.path.join(dir_name, filename),
                "batch_idx": idx,
            })
            item = {
                'target': pred,
                key: batch[key],
                'modes': batch['modes'],
                'id': batch["id"],
            }
            if padding_mask is not None:
                item['padding_mask'] = padding_mask
            torch.save(item, os.path.join(kd_targets_path, filename))

    index = {
        'cfg': OmegaConf.to_container(cfg),
        'model_state_dict': cfg.model_state_dict,
        'index': index_items,
    }

    with open(os.path.join(kd_targets_path, 'index.json'), 'w', encoding='utf-8') as f:
        json.dump(index, f)

    logger.info(f'Knowledge-Distillation (KD) targets successfully created under: {kd_targets_path}')

In [5]:
extract_targets(cfg=cfg)

Exception: 